In [15]:
import numpy as np
import pandas as pd
import scipy.io as io
# from matplotlib import pyplot as plt
import mne
# from mne.minimum_norm import read_inverse_operator, compute_source_psd_epochs

# Pre-processing

In [16]:
# load files
data_dir = 'C:/Users/Lenovo/Desktop/NMA2022/Project/fingerflex 9 patients data/fingerflex/data/'
pat = 'bp'
alldat = io.loadmat(data_dir + '/' + pat + '/' + pat +'_fingerflex.mat')
allresp = io.loadmat(data_dir + '/' + pat + '/' + pat +'_stim.mat')
alldat['resp'] = allresp['stim']

In [17]:
#print data shape
print(alldat['__header__'])
print(alldat['__version__'])
print(alldat['__globals__'])
print('data:', type(alldat['data']), alldat['data'].shape)
print('elec_regions:', type(alldat['elec_regions']),alldat['elec_regions'].shape)
print('flex:', type(alldat['flex']),alldat['flex'].shape)
print('brain:', type(alldat['brain']),alldat['brain'].shape)
print('locs:', type(alldat['locs']),alldat['locs'].shape)
print('cue:', type(alldat['cue']),alldat['cue'].shape)
print('response:', type(alldat['resp']),alldat['resp'].shape)

b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Mon Jan  4 09:05:12 2016'
1.0
[]
data: <class 'numpy.ndarray'> (610040, 46)
elec_regions: <class 'numpy.ndarray'> (46, 1)
flex: <class 'numpy.ndarray'> (610040, 5)
brain: <class 'numpy.ndarray'> (1, 1)
locs: <class 'numpy.ndarray'> (46, 3)
cue: <class 'numpy.ndarray'> (610040, 1)
response: <class 'numpy.ndarray'> (610040, 1)


In [ ]:
!pip install nilearn --quiet
!pip install nimare --quiet

from nilearn import plotting
from nimare import utils

In [ ]:

plt.figure(figsize=(8, 8))
locs = alldat['locs']
view = plotting.view_markers(utils.tal2mni(locs),
                             marker_labels=['%d'%k for k in np.arange(locs.shape[0])],
                             marker_color='purple',
                             marker_size=5)
view

In [18]:
sampling_freq = 1000  # in Hertz
elect_region  =alldat['elec_regions']
n_channels = len(elect_region)
ch_types = 'ecog'
ecog = alldat['data'].T

info = mne.create_info(n_channels, ch_types = 'ecog', sfreq = sampling_freq)
raw = mne.io.RawArray(ecog, info)
info

Creating RawArray with float64 data, n_channels=46, n_times=610040
    Range : 0 ... 610039 =      0.000 ...   610.039 secs
Ready.


<Info | 7 non-empty values
 bads: []
 ch_names: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, ...
 chs: 46 ECoG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 46
 projs: []
 sfreq: 1000.0 Hz
>

In [19]:
#filter
raw_filter = raw.copy().filter(l_freq=1., h_freq=None, picks='ecog') #1-200Hz
raw_notch = raw_filter.copy().notch_filter(freqs=60, picks='ecog') # 60Hz
# raw_downsampled = raw_notch.copy().resample(sfreq=200)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 3301 samples (3.301 sec)

Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 6601 samples (6.601 sec)



In [22]:
# raw_notch.pick(picks='all')
# dict_r=ecog(1000) scalings=dict_r
# raw_notch.plot()
# plt.show

In [23]:
new_ecog = raw_notch.get_data()
print(new_ecog.shape)
ecog = new_ecog
elect_region = alldat['elec_regions']
print(len(ecog))

(46, 610040)
46


In [24]:
#exclude resp=-1, -2
resp = alldat['resp']
resp[resp == -1] = 0
resp[resp == -2] = 0

cue = alldat['cue']
np.unique(cue)

array([0, 1, 2, 3, 4, 5], dtype=uint8)

# Trails extraction

In [25]:
# trails extraction (based on the one-to-one corresponding relation of each response and stimuli)
def trail_extract(cue, resp):

  for i in range(0, len(cue)):

    if i == 0:
      cue_mat = np.array([cue[0,0],0,0])
      resp_mat = np.array([resp[0,0],0])
      
    elif cue[i] != cue[i-1] and cue[i] != 0:
      last_cue = np.array([cue[i,0],i,i+4000])

      if i+4000 < len(resp):

        for t in range(i,i+2000):

          if resp[i] != 0:
            if resp[t] != resp[t-1] and resp[t] != 0:
              this_resp = np.array([resp[t,0], t])
              resp_mat = np.vstack((resp_mat, this_resp))   
              cue_mat = np.vstack((cue_mat, last_cue))

              break
            elif resp[t] == 0 and (not np.any(resp[t:i+2000])):
              for t_a in range(i+2000,i+4000):
                if resp[t_a] != 0:
                  this_resp = np.array([resp[t_a,0], t-a])
                  resp_mat = np.vstack((resp_mat, this_resp))
                  cue_mat = np.vstack((cue_mat, last_cue))   

                break
              break

          else:
            if resp[t] != 0:
              this_resp = np.array([resp[t,0], t])
              resp_mat = np.vstack((resp_mat, this_resp))   
              cue_mat = np.vstack((cue_mat, last_cue))

              break

            elif not np.any(resp[t:i+2000]):
              for t_a in range(i+2000,i+4000):
                if resp[t_a] != 0:
                  # print(i+2000, i+4000)
                  # print(t_a)
                  this_resp = np.array([resp[t_a,0], t-a])
                  resp_mat = np.vstack((resp_mat, this_resp)) 
                  cue_mat = np.vstack((cue_mat, last_cue))  
                break
              break


# if there is no response inside, we need to check the interval

  return cue_mat[1:,:], resp_mat[1:,:]

cue_mat, resp_mat = trail_extract(cue, resp)

In [26]:
print(cue_mat.shape)

(144, 3)


In [27]:
# correct & wrong extraction
a = 0

# print(cue_mat.shape)
# print(resp_mat.shape)

wrong_trial = np.append(cue_mat[0,:], resp_mat[0,:])
correct_trial = np.append(cue_mat[0,:], resp_mat[0,:])
label = np.empty([len(resp_mat)]) 

for i in range(0,resp_mat.shape[0]):
  if cue_mat[i,0] != resp_mat[i,0]:
    new_wrong = np.append(cue_mat[i,:], resp_mat[i,:])
    wrong_trial = np.vstack((wrong_trial, new_wrong))
    label[i] = 1
    a += 1
  else:
    new_correct = np.append(cue_mat[i,:], resp_mat[i,:])
    correct_trial = np.vstack((correct_trial, new_correct))
    label[i] = 0

wrong_trial = wrong_trial[1:]
correct_trial = correct_trial[1:]

rt_wrong = wrong_trial[:,4] - wrong_trial[:,1]
rt_correct = correct_trial[:,4] - correct_trial[:,1]

# print(wrong_trial[:3])
# print(correct_trial[:3])
# print(rt_correct)
# print(correct_trial)

In [32]:
# correct trails' cue and resp array
c_cue = correct_trial[:,0:3]
c_resp = correct_trial[:,3:len(correct_trial)]
print(c_cue)
print(c_resp.shape)

[[     5   7080  11080]
 [     5  11080  15080]
 [     5  15080  19080]
 [     1  19080  23080]
 [     5  23080  27080]
 [     2  27080  31080]
 [     3  31080  35080]
 [     4  35080  39080]
 [     1  39080  43080]
 [     1  43080  47080]
 [     1  47080  51080]
 [     5  51080  55080]
 [     2  55080  59080]
 [     4  59080  63080]
 [     3  63080  67080]
 [     2  67080  71080]
 [     3  71080  75080]
 [     5  75080  79080]
 [     2  79080  83080]
 [     2  83080  87080]
 [     1  91080  95080]
 [     4  95080  99080]
 [     3  99080 103080]
 [     1 103080 107080]
 [     4 107080 111080]
 [     1 111080 115080]
 [     3 115080 119080]
 [     3 119080 123080]
 [     1 123080 127080]
 [     4 127080 131080]
 [     2 131080 135080]
 [     3 135080 139080]
 [     5 139080 143080]
 [     2 143080 147080]
 [     5 147080 151080]
 [     4 151080 155080]
 [     4 155080 159080]
 [     3 159080 163080]
 [     2 163080 167080]
 [     3 167080 171080]
 [     1 175080 179080]
 [     5 179080 

In [144]:
#trails for each finger extraction
print('CueOnset','RespOnset')
def finger(value):
    #input: 1-5
    #output: finger mat with relevant cue and resp mat data + diff calculation 
#     finger_mat = [0,0,0,0] #fake first row
#     finger_mat = [0,0,0,0,0,0]
#     print('Finger','CueOnset','EPE','Finger','RespOnset','RT')
    #EPE = effective period end
    
    onset_c_finger = [0,0] 
    
    for i in range(len(c_cue)):
      if c_cue[i][0]==value:
        c_row = c_cue[i]
        r_row = c_resp[i]
#         diff = c_resp[i][1] - c_cue[i][1]
        row = np.concatenate((c_row[1],r_row[1]),axis=None)
        onset_c_finger = np.vstack((onset_c_finger,row))
    onset_c_finger = np.delete(onset_c_finger, 0, 0)
    
#         row = np.concatenate((c_row,r_row,diff),axis=None)
#         finger_mat=np.vstack((finger_mat,row))
#     finger_mat=np.delete(finger_mat, 0, 0)
    return onset_c_finger

# print(finger(1))
# print(finger(2).shape)
# print(finger(3).shape)
# print(finger(4).shape)
# print(finger(5).shape)

CueOnset RespOnset


In [163]:
# RT for each finger
rt_f1 = finger(1)[:,1] - finger(1)[:,0]
rt_f2 = finger(2)[:,1] - finger(2)[:,0]
rt_f3 = finger(3)[:,1] - finger(3)[:,0]
rt_f4 = finger(4)[:,1] - finger(4)[:,0]
rt_f5 = finger(5)[:,1] - finger(5)[:,0]

# plt.plot(rt_f1)
# plt.plot(rt_f2)
# plt.plot(rt_f3)
# plt.plot(rt_f4)
# plt.plot(rt_f5)
# plt.legend(['Thumb', 'Index', 'Middle', 'Fourth', 'Little'])
# plt.title('Response Time')
# plt.show()

rt_f5



array([1172,  822,  610,  812,  894,  828,  811,  795,  861, 1109, 1060,
        993, 1324,  911, 1093,  894,  828,  713,  712,  861,  564, 1307,
        977,  960,  778,  779,  944,  911])

In [166]:
#trails for each finger extraction 2
print('CueOnset','RespOnset')
def fingerflex(value):
    #input: 1-5
    #output: finger mat with relevant cue and resp mat data + diff calculation 
#     finger_mat = [0,0,0,0] #fake first row
    finger_mat = [0,0]
#     print('Finger','CueOnset','EPE','Finger','RespOnset','RT')
    #EPE = effective period end
    
#     onset_c_finger = [0,0] 
    
    for i in range(len(c_cue)):
      if c_cue[i][0]==value:
        c_row = c_cue[i]
        r_row = c_resp[i]
#         diff = c_resp[i][1] - c_cue[i][1]
#         row = np.concatenate((c_row[1],r_row[1]),axis=None)
#         onset_c_finger = np.vstack((onset_c_finger,row))
#     onset_c_finger = np.delete(onset_c_finger, 0, 0)
    
        row = np.concatenate((c_row[1], c_row[0]),axis=None)
        finger_mat=np.vstack((finger_mat,row))
    finger_mat=np.delete(finger_mat, 0, 0)
    return finger_mat

CueOnset RespOnset


In [175]:
# events extraction for eeglab
event_f1=fingerflex(1)
events_f1 = np.column_stack((event_f1[:,0], np.full((len(event_f1), 1), 'cue'), event_f1[:,1]))
events_f1
with open("event_f1.txt", "w") as o:
    for line in events_f1:
        print("{} {} {}".format(line[0], line[1], line[2]), file=o)
        
event_f2=fingerflex(2)
events_f2 = np.column_stack((event_f2[:,0], np.full((len(event_f2), 1), 'cue'), event_f2[:,1]))
events_f2
with open("event_f2.txt", "w") as o:
    for line in events_f2:
        print("{} {} {}".format(line[0], line[1], line[2]), file=o)
        
event_f3=fingerflex(3)
events_f3 = np.column_stack((event_f3[:,0], np.full((len(event_f3), 1), 'cue'), event_f3[:,1]))
events_f3
with open("event_f3.txt", "w") as o:
    for line in events_f3:
        print("{} {} {}".format(line[0], line[1], line[2]), file=o)
        
event_f4=fingerflex(4)
events_f4 = np.column_stack((event_f4[:,0], np.full((len(event_f4), 1), 'cue'), event_f4[:,1]))
events_f4
with open("event_f4.txt", "w") as o:
    for line in events_f4:
        print("{} {} {}".format(line[0], line[1], line[2]), file=o)
        
event_f5=fingerflex(5)
events_f5 = np.column_stack((event_f5[:,0], np.full((len(event_f5), 1), 'cue'), event_f5[:,1]))
events_f5
with open("event_f5.txt", "w") as o:
    for line in events_f5:
        print("{} {} {}".format(line[0], line[1], line[2]), file=o)

# PSD-freq and RT correlation for each finger flexion

In [ ]:
import scipy.signal as signal
import os

In [ ]:
def ecog_region(region_index, ecog):
  # 1: dorsal m1, 3: dorsal s1, 4: ventral sensorimotor (m1+s1)
  # 6: frontal (higher cognitive functions such as memory, emotions, impulse control, problem solving, social interaction, and motor function); 
  # 7: parietal (process sensory info), 
  # 8:temporal (object recognition, interact to creat new and long term memories), 9: occipital (visual perception)
#   print(elect_region)
  region = np.where(elect_region == region_index)[0]

  region_ecog = ecog[region]

  return region, region_ecog
print(ecog_region(1,ecog1))

In [ ]:
def w_c_extraction(elec_list, ecog_elec):

  ecog_wrong = np.empty([a,len(elec_list),2000])
  ecog_correct = np.empty([resp_mat.shape[0] - a,len(elec_list),2000])  


  for i in range(0,wrong_trial.shape[0]):
    onset_t = wrong_trial[i,1]
    # offset_t = onset_t + 2000
    ecog_wrong[i] = ecog_elec[:,onset_t:onset_t+2000]
    

  for i in range(0,correct_trial.shape[0]):
    onset_t = correct_trial[i,1] 
    # offset_t = onset_t + 2000
    ecog_correct[i] = ecog_elec[:,onset_t:onset_t+2000]

  return ecog_wrong, ecog_correct

In [ ]:
def psd_calculation(ecog_wrong, ecog_correct):

  w_f, wrong_psd = signal.welch(ecog_wrong, fs = 1000, nperseg= 1024, noverlap = 1024//2)
  c_f, correct_psd = signal.welch(ecog_correct, fs = 1000, nperseg= 1024, noverlap = 1024//2)


  # alpha_index = np.where((w_f<25) & (w_f>13))[0]

  # w_f = w_f[alpha_index]
  # c_f = c_f[alpha_index]

  # wrong_psd = wrong_psd[:,:,alpha_index]
  # correct_psd = correct_psd[:,:,alpha_index]

  wrong_psd = 10*np.log10(wrong_psd)
  wrong_psd_ave = np.mean(wrong_psd, axis = 0)
  correct_psd = 10*np.log10(correct_psd)
  correct_psd_ave = np.mean(correct_psd, axis = 0)
  
  return w_f, wrong_psd_ave, correct_psd_ave

In [ ]:
def psd_extraction(index, ecog):

  elec, ecog = ecog_region(index, ecog)

  ecog_wrong, ecog_correct = w_c_extraction(elec, ecog)

  f, wrong_psd, correct_psd = psd_calculation(ecog_wrong, ecog_correct)

  return f, wrong_psd, correct_psd

In [ ]:
region_list = [1,4,6,7,8,9]
f,_,_ = psd_extraction(region_list[1], ecog)
diff_psd = np.empty([len(region_list),len(f)])

for i in range(len(region_list)):
  f, wrong_psd, correct_psd = psd_extraction(region_list[i], ecog)
  plt.plot(f, wrong_psd-correct_psd)
  diff_psd[i] = wrong_psd-correct_psd

plt.legend(['dorsal M1', 'ventral sensorimotor (M1+S1)', 'frontal', 'parietal','temporal','occipital'])
plt.title('alpha ocssilation')
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [dB]')
plt.show()

# Motor preparation pattern examination

In [141]:
# minimum RT extraction
rt_all = correct_trial[:,4] - correct_trial[:,1]
rtmin_all = rt_all.min()
print(rtmin_all)

1870


Construct epochs based on rt_all.min and respOnset

In [170]:
# events creation for mne
event_id = {
    'Thumb': 1,
    'Index': 2, 
    'Middle': 3, 
    'Fourth': 4,
    'Little': 5
}

array_0 = np.zeros((len(correct_trial),1))
array_0.dtype

events = np.column_stack((correct_trial[:,4], array_0, correct_trial[:,3]))
# events=events.astype(np.uint8)
events.dtype
events = events.astype('int')
# raw_notch.plot(events=events, event_id=event_id)
# plt.show

metadata, events, event_id = mne.epochs.make_metadata(
    events=all_events, event_id=all_event_id,
    tmin=metadata_tmin, tmax=metadata_tmax, sfreq=raw_notch.info['sfreq'])

dtype('float64')

In [139]:
def txt2array(txt_path, delimiter):
    #---
    # 功能：读取只包含数字的txt文件，并转化为array形式
    # txt_path：txt的路径；delimiter：数据之间的分隔符
    #---
    data_list = []
    with open(txt_path) as f:
        data = f.readlines()
    for line in data:
        line = line.strip("\n")  # 去除末尾的换行符
        data_split = line.split(delimiter)
        temp = list(map(float, data_split))
        data_list.append(temp)

    data_array = np.array(data_list)
    return data_array


In [140]:
# locs for eeglab

aa=[]
for i in range(46):
  aa.append([])
  for j in range(1):
    aa[i].append(i)
bb=np.array(aa)

locs_bp_mat=txt2array('C:/Users/Lenovo/mne/locs_bp.txt',' ')
locs_new=np.column_stack((bb, locs_bp, bb))
locs_new

NameError: name 'string' is not defined

In [ ]:
#events distribution
mne.viz.plot_events(events)
plt.show

In [ ]:
# epochs creation for mne

rtmin_epochs = mne.Epochs(raw_notch, 
#                                info,
                               tmin=-rtmin_all,
#                                preload=True,
                               events=events, 
                               event_id=event_id,
                         preload=True)
rtmin_epochs

In [ ]:
rtmin_epochs.plot()

In [ ]:
# import matplotlib.pyplot as pyplt
# fig,ax = pyplt.subplots(2)

matplotlib.use('Qt5Agg')
raw_notch.plot_psd()
plt.show()